In [6]:
from backtesting.test import EURUSD, SMA
import pandas as pd
data = EURUSD.copy()


In [7]:
import numpy as np

def create_sequences(data, sequence_length=30, prediction_steps=5):
    X, y = [], []
    data = data.dropna().reset_index(drop=True)
    for i in range(sequence_length, len(data) - prediction_steps + 1):
        X.append(data[['Close']].iloc[i-sequence_length:i].values)
        y.append(data['Close'].pct_change(prediction_steps).iloc[i + prediction_steps - 1])
    return np.array(X), np.array(y)


def get_X(data):
    """Return model design matrix X"""
    return data.filter(like='X').values


def get_y(data):
    """Return dependent variable y"""
    y = data.Close.pct_change(48).shift(-48)  # Returns after roughly two days
    y[y.between(-.004, .004)] = 0             # Devalue returns smaller than 0.4%
    y[y > 0] = 1
    y[y < 0] = -1
    return y


def get_clean_Xy(df):
    """Return (X, y) cleaned of NaN values"""
    X = get_X(df)
    y = get_y(df).values
    isnan = np.isnan(y)
    X = X[~isnan]
    y = y[~isnan]
    return X, y

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

def build_model(input_shape):
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(50),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model



In [9]:
X, y = create_sequences(data, 30, 1)
X_train, y_train = X[:300], y[:300]

print(X, y)

[[[1.07219]
  [1.0726 ]
  [1.07192]
  ...
  [1.07506]
  [1.07659]
  [1.07657]]

 [[1.0726 ]
  [1.07192]
  [1.07202]
  ...
  [1.07659]
  [1.07657]
  [1.07574]]

 [[1.07192]
  [1.07202]
  [1.0705 ]
  ...
  [1.07657]
  [1.07574]
  [1.07458]]

 ...

 [[1.23962]
  [1.24174]
  [1.24028]
  ...
  [1.2339 ]
  [1.23501]
  [1.23422]]

 [[1.24174]
  [1.24028]
  [1.24088]
  ...
  [1.23501]
  [1.23422]
  [1.23372]]

 [[1.24028]
  [1.24088]
  [1.23726]
  ...
  [1.23422]
  [1.23372]
  [1.23426]]] [-0.00077097 -0.00107833 -0.00256845 ... -0.00040511  0.0004377
 -0.00422925]


In [5]:
%%time

import pandas as pd

from backtesting import Backtest, Strategy

class LSTMStrategy(Strategy):
    n_train = 300

    def init(self):
        print("Initializing strategy...")
        # Ensure data is DataFrame and add 'returns' column
        if isinstance(self.data.df, pd.DataFrame):
            self.data.df['returns'] = self.data.df['Close'].pct_change()

            # Prepare data for LSTM
            print("Creating sequences...")
            self.X, self.y = create_sequences(self.data.df, 30, 5)
            self.X_train, self.y_train = self.X[:self.n_train], self.y[:self.n_train]

            # Build and train the model
            print("Building and training the model...")
            self.model = build_model((30, 1))
            self.model.fit(self.X_train, self.y_train, epochs=5, batch_size=64, verbose=1)
            print("Model training completed.")
            self.model.save("lstmTest.h5")

    def next(self):
        print(f"Processing next step for data at index {len(self.data)}...")
        if len(self.data) < self.n_train + 30:
            print("Not enough data to predict. Skipping...")
            return

        # Get the latest data sequence for prediction
        X_last = self.data.df['Close'][-30:].values.reshape(1, 30, 1)
        prediction = self.model.predict(X_last)[0][0]
        print(f"Prediction: {prediction}")

        # Basic trading logic based on the prediction
        if prediction > 0:
            print("Predicted an upward movement. Placing a buy order.")
            self.buy()
        elif prediction < 0:
            print("Predicted a downward movement. Placing a sell order.")
            self.sell()



bt = Backtest(data, LSTMStrategy, commission=.0002, margin=.05, exclusive_orders=True)
results = bt.run()


Initializing strategy...
Creating sequences...


KeyboardInterrupt: 

In [ ]:
print(results)

Start                     2017-04-25 12:00:00
End                       2018-02-07 15:00:00
Duration                    288 days 03:00:00
Exposure Time [%]                   54.295042
Equity Final [$]                     0.057564
Equity Peak [$]                       10000.0
Return [%]                         -99.999424
Buy & Hold Return [%]               12.869869
Return (Ann.) [%]                  -99.999571
Volatility (Ann.) [%]                0.000751
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -99.999425
Avg. Drawdown [%]                  -99.999425
Max. Drawdown Duration      268 days 10:00:00
Avg. Drawdown Duration      268 days 10:00:00
# Trades                                 2658
Win Rate [%]                        34.875847
Best Trade [%]                       1.009808
Worst Trade [%]                     -0.695827
Avg. Trade [%]                    

In [1]:
bt.plot()

NameError: name 'bt' is not defined